In [2]:
using Pkg

Pkg.activate("..")

using SymPy

include("Helper_scripts/julia/SymPy.Jl")

 Activating environment at `~/Projects/Working/Bacterial_Interactions/Project.toml`


#89 (generic function with 1 method)

In [85]:
C1,C2 = symbols("C_1,C_2")
r1,r2 = symbols("r_1,r_2")
a11,a12,a21,a22,a = symbols("a_11,a_12,a_21,a_22,alpha")
b = symbols("beta")
Ktot,dK,K1,K2 = symbols("K_tot,Delta_K,K_1,K_2")

f1 = (r1 - a11 * C1 + a*C2)
f2 = (r2 - a22 * C2 + a*C1)

x = solve(f1,C2)[1]
y = solve(f2,C1)[1]

C1_sol = solve(f2.subs(C2,x),C1)[1]
C2_sol = solve(f1.subs(C1,y),C2)[1]


[C1_sol, C2_sol]

2-element Array{Sym,1}:
 (a_22*r_1 + alpha*r_2)/(a_11*a_22 - alpha^2)
 (a_11*r_2 + alpha*r_1)/(a_11*a_22 - alpha^2)

In [99]:
Y = C2_sol.args[1].args[1]
X = symbols("x")

f = (C1_sol + C2_sol).subs(Y,1) - Ktot*X

a_poly = collect(expand(f.subs(X,bot)),a)


                      2                                
-Kₜₒₜ⋅a₁₁⋅a₂₂ + Kₜₒₜ⋅α  + a₁₁⋅r₂ + a₂₂⋅r₁ + α⋅(r₁ + r₂)

In [100]:
roots = SymPy.solve(a_poly,a)
roots

2-element Array{Sym,1}:
 (-r_1 - r_2 + sqrt(4*K_tot^2*a_11*a_22 - 4*K_tot*a_11*r_2 - 4*K_tot*a_22*r_1 + r_1^2 + 2*r_1*r_2 + r_2^2))/(2*K_tot)
 -(r_1 + r_2 + sqrt(4*K_tot^2*a_11*a_22 - 4*K_tot*a_11*r_2 - 4*K_tot*a_22*r_1 + r_1^2 + 2*r_1*r_2 + r_2^2))/(2*K_tot)

In [33]:
# Feasabilty


             2      ⎛           2⎞                              
-a₁₁⋅α⋅r₂ + α ⋅r₁ + ⎝a₁₁⋅a₂₂ - α ⎠⋅(-a₁₁⋅a₂₂⋅r₁ + a₁₁⋅α⋅r₂ + r₁)
────────────────────────────────────────────────────────────────
                                     2                          
                          a₁₁⋅a₂₂ - α                           

# Stability

In [10]:
#get jacobian
J = [(f1).diff(C1) (f1).diff(C2) ; (f2).diff(C1) (f2).diff(C2)]
J_eq = map(x -> subs(x,(C1,C1_sol),(C2,C2_sol)).simplify(), J)



2×2 Array{Sym,2}:
  -a_11*(a_22*r_1 - alpha*r_2)/(a_11*a_22 - alpha^2)  …  -alpha*(a_22*r_1 - alpha*r_2)/(a_11*a_22 - alpha^2)
 -alpha*(a_11*r_2 - alpha*r_1)/(a_11*a_22 - alpha^2)      -a_22*(a_11*r_2 - alpha*r_1)/(a_11*a_22 - alpha^2)

In [20]:
a,b,c,d = symbols("a,b,c,d")

expand(((a+d)^2 + sqrt( (a+d)^2 - 4*(a*d - b*c)))^2)

                                _________________________                     
 4      3        2  2      2   ╱  2                    2     2        3       
a  + 4⋅a ⋅d + 6⋅a ⋅d  + 2⋅a ⋅╲╱  a  - 2⋅a⋅d + 4⋅b⋅c + d   + a  + 4⋅a⋅d  + 4⋅a⋅

     _________________________                                ________________
    ╱  2                    2                     4      2   ╱  2             
d⋅╲╱  a  - 2⋅a⋅d + 4⋅b⋅c + d   - 2⋅a⋅d + 4⋅b⋅c + d  + 2⋅d ⋅╲╱  a  - 2⋅a⋅d + 4⋅

_________     
       2     2
b⋅c + d   + d 